<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS_2B/blob/main/Copy_of_DL_ASS2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key='c118c1c1779ae69dc5db59f0ff99465d34292bfe')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 3628-pavitrakhare (3628-pavitrakhare-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision import models

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define function for pretrain model with multiple strategies
def pretrain_model(model_name, strategy):
    """
    Initialize a pretrained model with different freezing strategies:
    - strategy='freeze_all_except_last': Freeze all layers except the output layer
    - strategy='freeze_80_percent': Freeze 80% of the layers from the beginning
    - strategy='freeze_fc_only': Freeze only the fully connected layers
    - strategy='train_from_scratch': Initialize the model but don't use pretrained weights
    """

    if model_name == 'resnet':
        if strategy == 'train_from_scratch':
            pretrained_model = models.resnet50(pretrained=False)
        else:
            pretrained_model = models.resnet50(pretrained=True)
    elif model_name == 'inception':
        if strategy == 'train_from_scratch':
            pretrained_model = models.inception_v3(pretrained=False)
        else:
            pretrained_model = models.inception_v3(pretrained=True)

    # Get number of features in the final fully connected layer
    nodes_fc = pretrained_model.fc.in_features

    # Replace the final layer to match our number of classes (10)
    pretrained_model.fc = nn.Linear(nodes_fc, 10)

    # Apply different freezing strategies
    if strategy == 'freeze_all_except_last':
        # Freeze all layers except the final fully connected layer
        for name, parameter in pretrained_model.named_parameters():
            if not name.startswith('fc'):
                parameter.requires_grad = False

    elif strategy == 'freeze_80_percent':
        # Freeze 80% of the layers from the beginning
        total_layers = sum(1 for _ in pretrained_model.children())
        num_freeze = int(total_layers * 0.8)

        count = 0
        for name, child in pretrained_model.named_children():
            if count < num_freeze:
                for param in child.parameters():
                    param.requires_grad = False
            count += 1

    elif strategy == 'freeze_fc_only':
        # Freeze only the fully connected layers (reverse of typical strategy)
        # In ResNet, this means freezing the fc layer but training all conv layers
        for name, parameter in pretrained_model.named_parameters():
            if name.startswith('fc'):
                parameter.requires_grad = False

    elif strategy == 'train_from_scratch':
        # All parameters are already trainable by default
        pass

    return pretrained_model

In [ ]:

data_dir = '/kaggle/input/my-dataset/inaturalist_12K/train'

def data_load(data_dir,data_augumentation):
    # Define data transformations
    if data_augumentation == 'Yes':
        transform = transforms.Compose([
            transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
            transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
            transforms.RandomRotation(20),  # Randomly rotate the image by up to 20 degrees
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])

    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Resize images to 224x224 (compatible with the CNN input size)
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image tensors
        ])

    # Load the dataset using ImageFolder and apply transformations
    training_data = ImageFolder(root=data_dir, transform=transform)
    # Splitting train dataset into training and validation indices
    train_index, val_index = train_test_split(list(range(len(training_data))), test_size=0.2, random_state=42)
    # Create DataLoader instances for training and validation sets
    random_train_sample = SubsetRandomSampler(train_index)
    train_data = DataLoader(
    training_data,
    batch_size=32,
    sampler=random_train_sample,
    num_workers=4,  # Add workers for parallel loading
    pin_memory=True  # Faster data transfer to GPU
    )

    random_val_sample = SubsetRandomSampler(val_index)
    validation_data = DataLoader(
    training_data,
    batch_size=32,
    sampler=random_val_sample,
    num_workers=4,  # Add workers for parallel loading
    pin_memory=True  # Faster data transfer to GPU
    )
    return train_data, validation_data

In [ ]:
def train_on_train_data(model, train_data):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()  # Set the model to training mode
    training_loss = 0.0
    correct_train_label = 0
    total_train = 0
    # Training loop
    for inputs, labels in train_data:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.item()

        # Calculate training accuracy
        _, pred = torch.max(outputs, 1)
        correct_train_label += (pred == labels).sum().item()
        total_train += labels.size(0)

    # Calculate training loss and accuracy
    avg_loss = training_loss / len(train_data)
    train_accuracy = 100 * correct_train_label / total_train
    return avg_loss, train_accuracy



In [ ]:
def test_on_valid_data(model, test_data):
    model.eval()  # Set the model to evaluation mode
    correct_val = 0
    total_val = 0
    with torch.no_grad():  # Turn off gradient calculation for validation
        for inputs_val, labels_val in test_data:
            inputs_val, labels_val = inputs_val.to(device), labels_val.to(device)
            outputs_val = model(inputs_val)
            _, predicted_val = torch.max(outputs_val, 1)
            correct_val += (predicted_val == labels_val).sum().item()
            total_val += labels_val.size(0)

    # Calculate validation accuracy
    valid_accuracy = 100 * correct_val / total_val
    return valid_accuracy

In [ ]:
# Add a function to load test data
def test_data_load(test_dir, data_augumentation='No'):
    # Define data transformations (same as in data_load function)
    if data_augumentation == 'Yes':
        transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    # Load the test dataset
    test_data = ImageFolder(root=test_dir, transform=transform)
    test_loader = DataLoader(
        test_data,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    return test_loader

In [ ]:
# Modify the model_train function to evaluate on test data after training
def model_train(model, train_data, val_data, test_data, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        avg_loss, train_accuracy = train_on_train_data(model, train_data)
        # Print training loss and accuracy
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        wandb.log({'Train loss': avg_loss})
        wandb.log({'Train accuracy': train_accuracy})

        # Validation loop
        val_accuracy = test_on_valid_data(model, val_data)
        # Print validation accuracy
        print(f'Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%')
        wandb.log({'val_accuracy': val_accuracy})
        wandb.log({'epoch': epoch})

    # After training is complete, evaluate on test data
    test_accuracy = test_on_valid_data(model, test_data)
    print(f'Test Accuracy: {test_accuracy:.2f}%')
    wandb.log({'test_accuracy': test_accuracy})

    print('Training complete!')

In [ ]:


# Update the sweep configuration to include the strategies
sweep_config = {
    'method': 'grid',  # Changed to grid to ensure all strategies are tested
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epoch': {
            'values': [10]
        },
        'strategy': {
            'values': [
                'freeze_all_except_last',  # Strategy 1
                'freeze_80_percent',       # Strategy 2
                'freeze_fc_only',          # Strategy 3
                'train_from_scratch'       # Strategy 4
            ]
        },
        'model': {
            'values': ['resnet']
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_ASS2_B')



# Update the main function to load test data and pass it to model_train
def main():
    with wandb.init() as run:
        # Update run name to include strategy
        run_name = f"ep{wandb.config.epoch}_strategy-{wandb.config.strategy}_model-{wandb.config.model}"
        wandb.run.name = run_name

        # Initialize model with the selected strategy
        model = pretrain_model(model_name=wandb.config.model, strategy=wandb.config.strategy)
        model = model.to(device)

        # Load training and validation data
        data_dir = '/kaggle/input/my-dataset/inaturalist_12K/train'
        train, validation = data_load(data_dir, data_augumentation='No')

        # Load test data
        test_dir = '/kaggle/input/my-dataset/inaturalist_12K/val'
        test_data = test_data_load(test_dir, data_augumentation='No')

        # Train model and evaluate on test data
        model_train(model, train, validation, test_data, epochs=wandb.config.epoch)

wandb.agent(sweep_id, function=main, count=4)  # Run all 4 strategies
wandb.finish()

Create sweep with ID: yg114ke1
Sweep URL: https://wandb.ai/3628-pavitrakhare-indian-institute-of-technology-madras/DL_ASS2_B/sweeps/yg114ke1


wandb: Agent Starting Run: pntzdxvl with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: freeze_all_except_last


Epoch 1/10, Train Loss: 1.1234, Train Accuracy: 64.22%
Epoch 1/10, Validation Accuracy: 72.05%
Epoch 2/10, Train Loss: 0.8553, Train Accuracy: 71.73%
Epoch 2/10, Validation Accuracy: 72.75%
Epoch 3/10, Train Loss: 0.7922, Train Accuracy: 73.58%
Epoch 3/10, Validation Accuracy: 73.35%
Epoch 4/10, Train Loss: 0.7371, Train Accuracy: 75.41%
Epoch 4/10, Validation Accuracy: 75.10%
Epoch 5/10, Train Loss: 0.7495, Train Accuracy: 75.11%
Epoch 5/10, Validation Accuracy: 73.55%
Epoch 6/10, Train Loss: 0.7172, Train Accuracy: 75.92%
Epoch 6/10, Validation Accuracy: 73.90%
Epoch 7/10, Train Loss: 0.6923, Train Accuracy: 76.81%
Epoch 7/10, Validation Accuracy: 73.85%
Epoch 8/10, Train Loss: 0.6931, Train Accuracy: 76.90%
Epoch 8/10, Validation Accuracy: 73.85%
Epoch 9/10, Train Loss: 0.6600, Train Accuracy: 78.03%
Epoch 9/10, Validation Accuracy: 71.90%
Epoch 10/10, Train Loss: 0.6697, Train Accuracy: 77.63%
Epoch 10/10, Validation Accuracy: 75.40%
Test Accuracy: 76.45%
Training complete!


Train accuracy,▁▅▆▇▇▇▇▇██
Train loss,█▄▃▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▄▇▄▅▅▅▁█
Train accuracy,77.6347
Train loss,0.66973
epoch,9
test_accuracy,76.45
val_accuracy,75.4


wandb: Agent Starting Run: fk1qcamn with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: freeze_80_percent


...

Train accuracy,▁▂▃▃▄▅▆▇▇█
Train loss,█▇▆▆▅▄▃▂▂▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▄▆▇▇▇▇█▇
Train accuracy,84.37305
Train loss,0.4705
epoch,9
test_accuracy,46.85
val_accuracy,45.8


wandb: Agent Starting Run: g8svasgz with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: train_from_scratch


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 2.3064, Train Accuracy: 19.74%
Epoch 1/10, Validation Accuracy: 20.15%
Epoch 2/10, Train Loss: 2.2084, Train Accuracy: 19.16%
Epoch 2/10, Validation Accuracy: 22.20%
Epoch 3/10, Train Loss: 2.1853, Train Accuracy: 20.55%
Epoch 3/10, Validation Accuracy: 20.80%
Epoch 4/10, Train Loss: 2.1428, Train Accuracy: 22.30%
Epoch 4/10, Validation Accuracy: 22.25%
Epoch 5/10, Train Loss: 2.1489, Train Accuracy: 22.23%
Epoch 5/10, Validation Accuracy: 23.30%
Epoch 6/10, Train Loss: 2.1172, Train Accuracy: 24.14%
Epoch 6/10, Validation Accuracy: 22.95%
Epoch 7/10, Train Loss: 2.0786, Train Accuracy: 25.50%
Epoch 7/10, Validation Accuracy: 23.85%
Epoch 8/10, Train Loss: 2.0630, Train Accuracy: 25.89%
Epoch 8/10, Validation Accuracy: 26.15%
Epoch 9/10, Train Loss: 2.0446, Train Accuracy: 26.42%
Epoch 9/10, Validation Accuracy: 25.40%
Epoch 10/10, Train Loss: 2.0132, Train Accuracy: 27.73%
Epoch 10/10, Validation Accuracy: 25.90%
Test Accuracy: 26.65%
Training complete!


Train accuracy,▁▁▂▄▄▅▆▆▇█
Train loss,█▆▅▄▄▃▃▂▂▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▂▃▅▄▅█▇█
Train accuracy,27.72847
Train loss,2.01322
epoch,9
test_accuracy,26.65
val_accuracy,25.9
